In [ ]:
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup

# def extract_short_summary(url):
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.content, 'html.parser')
#             # Targeting the specific structure of the Short Summary
#             summary_element = soup.find('span', id='Short_Summary')
#             if summary_element:
#                 summary_paragraph = summary_element.find_parent('h2').find_next_sibling('p')
#                 if summary_paragraph:
#                     return summary_paragraph.get_text(strip=True)
#         return "Summary not found or inaccessible"
#     except requests.RequestException as e:
#         return f"Error in accessing page: {e}"

# # Load the CSV file
# df = pd.read_csv('Greys_Anatomy_Episodes_Cleaned.csv')

# # Apply the function to each URL in the Title_Link column
# df['Short Summary'] = df['Title_Link'].apply(extract_short_summary)

# # Save the updated DataFrame to a new CSV file
# df.to_csv('Greys_Anatomy_Episodes_Cleaned_with_summaries.csv', index=False)


In [ ]:
# !pip install --upgrade langchain
# !pip install python-dotenv
# !pip install docarray
# !pip install tiktoken

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
llm_model = "gpt-4-1106-preview"

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'Greys_Anatomy_Episodes_Cleaned_with_summaries.csv'
loader = CSVLoader(file_path=file)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
query ="Please list all the episodes with Meredith"

Please list all the episodes
in which Derek kiss Meredith.

Please list all the episodes
in which Meredith cries.

Please list all the episodes
in which Callie cries.

Please list all the episodes
in which Owen returns to Seattle Grace.

Are Callie and Arizona in a romantic relationship?

With who is George in a romantic relationship?

Who is Olivia?

Who is Harper Avery?

Who is Sofia?


In [ ]:
response = index.query(query)

In [ ]:
display(Markdown(response))

In [5]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [6]:
docs = loader.load()

In [7]:
docs[0]

Document(page_content='Season: Season 1\nNo. in series: 1\nNo. in season: 1\nTitle_Text: "A Hard Day\'s Night"\nTitle_Link: https://greysanatomy.fandom.com/wiki/A_Hard_Day%27s_Night\nOriginal air date: March 27, 2005\nU.S. viewers (millions): 16.25\nShort Summary: Meredith Grey, Izzie Stevens, George O\'Malley, Cristina Yang, and Alex Karev become interns at Seattle Grace Hospital. Meredith discovers that her one night stand was actually with one of her bosses and she and her new friends discover that being a surgeon isn\'t as easy or fun as they thought it would be.', metadata={'source': 'Greys_Anatomy_Episodes_Cleaned_with_summaries.csv', 'row': 0})

In [8]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [9]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [10]:
print(len(embed))

1536


In [11]:
print(embed[:5])

[-0.021867523073664942, 0.006806864435505972, -0.018180996265900723, -0.03910486791222714, -0.014066680401294437]


In [12]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [13]:
query = "Please list all \
episodes about covid"

In [14]:
docs = db.similarity_search(query)

In [15]:
len(docs)

4

In [16]:
retriever = db.as_retriever()

In [17]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [18]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [23]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all \
episodes about covid in a table and report the summary. The table should be in markdown.") 

In [24]:
display(Markdown(response))

Here is a markdown table listing the episodes of "Grey's Anatomy" that specifically address the COVID-19 pandemic, along with their summaries:

```markdown
| Season | Episode | Title                     | Air Date       | U.S. Viewers (Millions) | Summary                                                                                                                                                                                                                   |
|--------|---------|---------------------------|----------------|-------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 17     | 1       | All Tomorrow's Parties    | November 12, 2020 | 5.93                    | In the midst of the early days of the COVID-19 pandemic, Grey Sloan Memorial and its surgeons' lives have been turned upside down. The season 17 premiere picks up one month into the pandemic, and it's all-hands-on-deck as Meredith, Bailey, and the rest of the Grey Sloan doctors find themselves on the frontlines of a new era. Meanwhile, an unintentionally started fire has first responders from Station 19 bringing patients into the hospital for treatment. |
| 17     | 2       | The Center Won't Hold     | November 12, 2020 | N/A                     | Bailey finds herself in the middle of an argument with patients' families as they await news about their children who were injured in a fire. Winston surprises Maggie, and Amelia and Link try to have some fun as new parents. Teddy learns her colleagues know more than she may like about her relationship woes with Owen.                                                                                                      |
| 17     | 3       | My Happy Ending           | November 19, 2020 | N/A                     | The Grey Sloan doctors continue to face their new COVID-19 reality and deal with a familiar and stubborn patient. Koracick is put in charge of the interns who recently joined the hospital ranks and Link operates on a sex therapist. Elsewhere, Maggie finds comfort in Winston's virtual presence.                                                                                                  |
| 17     | 4       | You'll Never Walk Alone   | December 3, 2020  | N/A                     | Owen faces a medical diagnosis that challenges more than he imagined. Koracick begins to go stir crazy. Maggie gets a not-so-subtle glimpse into Winston's background.                                                                                                                                                                                                 |
| 17     | 5       | Fight the Power           | December 10, 2020 | N/A                     | Bailey panics as she hears there has been a surge of COVID-19 cases, knowing she has loved ones in an assisted living facility. Meanwhile, Jackson and Richard team up against Catherine to teach her a lesson, and Teddy continues to try to mend her frayed relationships. After an intense surgery, Jo is uncertain about her future.                                                                 |
| 17     | 6       | No Time for Despair       | December 17, 2020 | N/A                     | Grey Sloan Memorial faces new pressures as Seattle Pres is overloaded, and Grey Sloan Memorial is now on surge capacity protocol. Meanwhile, Owen and Amelia are faced with one of the most controversial surgeries of their careers.                                                                                                                                  |
| 17     | 7       | Helplessly Hoping         | March 11, 2021    | N/A                     | The doctors are joined by the team from Station 19 as they tend to a tense situation with a lot at stake. Jo convinces Hayes to bend the rules on a case, and Maggie and Winston reconnect.                                                                                                                                                                             |
| 17     | 8       | It's All Too Much         | March 18, 2021    | N/A                     | As traumas and pressure mount, Grey Sloan doctors try to find a path forward, and Richard questions his faith. Meanwhile, Jo, Link, and Jackson play an unconventional drinking game.                                                                                                                                                                                     |
| 17     | 9       | In My Life                | March 25, 2021    | N/A                     | After a heart-wrenching loss, Teddy struggles to cope and it triggers memories of her past. When Owen brings her home, she must face some long-standing truths. Meanwhile, Amelia tries to help the situation by encouraging Owen to accept and forgive Teddy.                                                                                                             |
| 17     | 10      | Breathe                  | April 1, 2021      | N/A                     | The hospital ventilator shortage has the doctors gravely concerned when both a mother and daughter are in critical condition with COVID and fighting over the last ventilator. Meanwhile, Hayes' high-risk sister-in-law with multiple sclerosis ends up in the hospital with a kidney stone, and the sisters confront a long-held secret.                                                                 |
| 17     | 11      | Sorry Doesn't Always Make It Right | April 8, 2021      | N/A                     | The Grey Sloan doctors are stuck in the middle of a fight between a newlywed couple who are injured in a car accident and brought to the hospital for treatment. Meanwhile, Jackson's generosity with COVID-positive patients goes a bit too far, and Hayes works to rebuild Maggie's confidence and convince her to perform an extremely risky heart surgery.                                                                 |
| 17     | 12      | Sign O' the Times         | April 15, 2021    | N/A                     | While Meredith remains on a ventilator in the ICU, the team reflects on the impact of COVID-19 and the protests against police brutality. Meanwhile, Jackson pays a visit to his father that helps set him on the right path.                                                                                                                                              |
| 17     | 13      | Good as Hell              | April 22, 2021    | N/A                     | Amidst the need for more surgeons,